# 在Sagemaker上基于特征向量机算法实现电影推荐

## 实验步骤
* 实验介绍
* 数据准备
* 数据预处理+介绍Data Wrangler
* 模型训练
* 模型部署
* 模型推理

## 实验介绍

![架构图](https://github.com/VerRan/sagemaker-recommendation-workshop/blob/main/images/architecture.png?raw=true)

### 因子分解机算法介绍
该实验使用movielens，采用Sagemaker 托管的FM（因子分解机）算法实现推荐. 

![架构图](https://d-jcyfmfqr3kq8.studio.us-west-2.sagemaker.aws/jupyter/default/files/sagemaker-workshop/recomendation/images/FM-intro.png?_xsrf=2%7Cbe1f7bb5%7C33cd65b2b7d555958bf99ecfd95f43fb%7C1606445305)

主要考虑到多维特征（隐含特征）之间的交叉关系，其中参数的训练使用的是矩阵分解的方法。

### 下载 ml-100k 数据集
数据集说明：https://grouplens.org/datasets/movielens/

In [1]:
!wget http://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip -o ml-100k.zip

--2020-12-07 10:22:32--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  12.7MB/s    in 0.4s    

2020-12-07 10:22:33 (12.7 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base

## 数据分析
* 我们先来分析一下u.data的数据，这里包含的是用户的观影记录数据

In [58]:
import pandas as pd
data = pd.read_csv("u.data",sep='\t',header = None ,names=['userid','movieid','rating','timestamp'])
data.head

<bound method NDFrame.head of        userid  movieid  rating  timestamp
0         196      242       3  881250949
1         186      302       3  891717742
2          22      377       1  878887116
3         244       51       2  880606923
4         166      346       1  886397596
...       ...      ...     ...        ...
99995     880      476       3  880175444
99996     716      204       5  879795543
99997     276     1090       1  874795795
99998      13      225       2  882399156
99999      12      203       3  879959583

[100000 rows x 4 columns]>

* 下来我们看一下产品数据,产品信息列包含影片的信息

In [62]:
import pandas as pd
items = pd.read_csv("u.item",sep='|',header = None , encoding ="ISO-8859-1",usecols=[0,1,2,3,4,5,6,7,8,9,10])
items.head

<bound method NDFrame.head of          1                                   Toy Story (1995)  01-Jan-1995  \
0        2                                   GoldenEye (1995)  01-Jan-1995   
1        3                                  Four Rooms (1995)  01-Jan-1995   
2        4                                  Get Shorty (1995)  01-Jan-1995   
3        5                                     Copycat (1995)  01-Jan-1995   
4        6  Shanghai Triad (Yao a yao yao dao waipo qiao) ...  01-Jan-1995   
...    ...                                                ...          ...   
1676  1678                                  Mat' i syn (1997)  06-Feb-1998   
1677  1679                                   B. Monkey (1998)  06-Feb-1998   
1678  1680                               Sliding Doors (1998)  01-Jan-1998   
1679  1681                                You So Crazy (1994)  01-Jan-1994   
1680  1682          Scream of Stone (Schrei aus Stein) (1991)  08-Mar-1996   

      Unnamed: 3 http://us.imdb.c

* 下来我们看一下用户数据

In [64]:
import pandas as pd
users = pd.read_csv("u.user",sep='|',header = None , encoding ="ISO-8859-1",names=['userid','age','gender','occupation','zip'])
users.head

<bound method NDFrame.head of      userid  age gender     occupation    zip
0         1   24      M     technician  85711
1         2   53      F          other  94043
2         3   23      M         writer  32067
3         4   24      M     technician  43537
4         5   33      F          other  15213
..      ...  ...    ...            ...    ...
938     939   26      F        student  33319
939     940   32      M  administrator  02215
940     941   20      M        student  97229
941     942   48      F      librarian  78209
942     943   22      M        student  77841

[943 rows x 5 columns]>

In [2]:
%cd ml-100k
!shuf ua.base -o ua.base.shuffled
!head -10 ua.base.shuffled

/root/sagemaker-workshop/recomendation/dlnotebooks/sagemaker/ml-100k
606	179	5	880927552
727	423	3	883710830
207	156	2	878104438
563	220	4	880506703
758	154	5	881975267
774	185	2	888557683
85	506	4	886282959
696	9	5	886404617
303	155	3	879484159
125	209	4	879455241


数据包括userid,itemid,rating,timestamp 等属性

In [3]:
!head -10 ua.test

1	20	4	887431883
1	33	4	878542699
1	61	4	878542420
1	117	3	874965739
1	155	2	878542201
1	160	4	875072547
1	171	5	889751711
1	189	3	888732928
1	202	5	875072442
1	265	4	878542441


In [4]:
import sagemaker
import sagemaker.amazon.common as smac
from sagemaker import get_execution_role
from sagemaker.predictor import json_deserializer

import boto3, csv, io, json
import numpy as np
from scipy.sparse import lil_matrix

### 构建训练和测试数据集

In [5]:
nbUsers=943
nbMovies=1682
nbFeatures=nbUsers+nbMovies

nbRatingsTrain=90570
nbRatingsTest=9430

In [6]:
# For each user, build a list of rated movies.
# We'd need this to add random negative samples.
moviesByUser = {}
for userId in range(nbUsers):
    moviesByUser[str(userId)]=[]
 
with open('ua.base.shuffled','r') as f:
    samples=csv.reader(f,delimiter='\t')
    for userId,movieId,rating,timestamp in samples:
        moviesByUser[str(int(userId)-1)].append(int(movieId)-1) 

In [7]:
def loadDataset(filename, lines, columns):
    # Features are one-hot encoded in a sparse matrix
    X = lil_matrix((lines, columns)).astype('float32')
    # Labels are stored in a vector
    Y = []
    line=0
    with open(filename,'r') as f:
        samples=csv.reader(f,delimiter='\t')
        for userId,movieId,rating,timestamp in samples:
            X[line,int(userId)-1] = 1
            X[line,int(nbUsers)+int(movieId)-1] = 1
            if int(rating) >= 4:
                Y.append(1)
            else:
                Y.append(0)
            line=line+1
            
    Y=np.array(Y).astype('float32')
    return X,Y

In [8]:
X_train, Y_train = loadDataset('ua.base.shuffled', nbRatingsTrain, nbFeatures)
X_test, Y_test = loadDataset('ua.test',nbRatingsTest,nbFeatures)

In [9]:
print(X_train.shape)
print(Y_train.shape)
assert X_train.shape == (nbRatingsTrain, nbFeatures)
assert Y_train.shape == (nbRatingsTrain, )
zero_labels = np.count_nonzero(Y_train)
print("Training labels: %d zeros, %d ones" % (zero_labels, nbRatingsTrain-zero_labels))

print(X_test.shape)
print(Y_test.shape)
assert X_test.shape  == (nbRatingsTest, nbFeatures)
assert Y_test.shape  == (nbRatingsTest, )
zero_labels = np.count_nonzero(Y_test)
print("Test labels: %d zeros, %d ones" % (zero_labels, nbRatingsTest-zero_labels))

(90570, 2625)
(90570,)
Training labels: 49906 zeros, 40664 ones
(9430, 2625)
(9430,)
Test labels: 5469 zeros, 3961 ones


### 将csv转换为protobuf 存储到 S3

In [14]:
bucket = 'sagemaker-us-west-2-517141035927'
prefix = 'sagemaker/fm-movielens'

train_key      = 'train.protobuf'
train_prefix   = '{}/{}'.format(prefix, 'train3')

test_key       = 'test.protobuf'
test_prefix    = '{}/{}'.format(prefix, 'test3')

output_prefix  = 's3://{}/{}/output'.format(bucket, prefix)

In [15]:
def writeDatasetToProtobuf(X, Y, bucket, prefix, key):
    buf = io.BytesIO()
    smac.write_spmatrix_to_sparse_tensor(buf, X, Y)
    buf.seek(0)
    obj = '{}/{}'.format(prefix, key)
    boto3.resource('s3').Bucket(bucket).Object(obj).upload_fileobj(buf)
    return 's3://{}/{}'.format(bucket,obj)
    
train_data = writeDatasetToProtobuf(X_train, Y_train, bucket, train_prefix, train_key)    
test_data  = writeDatasetToProtobuf(X_test, Y_test, bucket, test_prefix, test_key)    
  
print(train_data)
print(test_data)
print('Output: {}'.format(output_prefix))

s3://sagemaker-us-west-2-517141035927/sagemaker/fm-movielens/train3/train.protobuf
s3://sagemaker-us-west-2-517141035927/sagemaker/fm-movielens/test3/test.protobuf
Output: s3://sagemaker-us-west-2-517141035927/sagemaker/fm-movielens/output


### 模型训练

In [16]:
containers = {'us-west-2': '174872318107.dkr.ecr.us-west-2.amazonaws.com/factorization-machines:latest',
              'us-east-1': '382416733822.dkr.ecr.us-east-1.amazonaws.com/factorization-machines:latest',
              'us-east-2': '404615174143.dkr.ecr.us-east-2.amazonaws.com/factorization-machines:latest',
              'eu-west-1': '438346466558.dkr.ecr.eu-west-1.amazonaws.com/factorization-machines:latest'}

In [17]:
fm = sagemaker.estimator.Estimator(containers[boto3.Session().region_name],
                                   get_execution_role(), 
                                   train_instance_count=1, 
                                   train_instance_type='ml.c4.xlarge',
                                   output_path=output_prefix,
                                   sagemaker_session=sagemaker.Session())

fm.set_hyperparameters(feature_dim=nbFeatures,
                      predictor_type='binary_classifier',
                      mini_batch_size=1000,
                      num_factors=64,
                      epochs=100)

fm.fit({'train': train_data, 'test': test_data})

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


2020-12-07 10:28:28 Starting - Starting the training job...
2020-12-07 10:28:30 Starting - Launching requested ML instances......
2020-12-07 10:29:40 Starting - Preparing the instances for training...
2020-12-07 10:30:25 Downloading - Downloading input data......
2020-12-07 10:31:24 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
Running default environment configuration script
/opt/amazon/lib/python2.7/site-packages/pandas/util/nosetester.py:13: DeprecationWarning: Importing from numpy.testing.nosetester is deprecated, import from numpy.testing instead.
  from numpy.testing import nosetester
[12/07/2020 10:31:26 INFO 140711371384640] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-conf.json: {u'factors_lr': u'0.0001', u'linear_init_sigma': u'0.01', u'epochs': 1, u'_wd': u'1.0', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'factors_init_sigma': u'0.001', u'

### 模型部署

In [22]:
fm_predictor = fm.deploy(instance_type='ml.c4.xlarge', initial_instance_count=1)

-------------!

In [33]:
fm_predictor.endpoint_name

'factorization-machines-2020-12-08-02-41-09-967'

In [49]:
def fm_serializer(data):
    js = {'instances': []}
    for row in data:
        js['instances'].append({'features': row.tolist()})
    print(js)
    return json.dumps(js)

fm_predictor.content_types = 'application/json'
fm_predictor.serializer =  sagemaker.serializers.JSONSerializer()
fm_predictor.deserializer = sagemaker.deserializers.JSONDeserializer()

### 模型推理

In [51]:
result = fm_predictor.predict(fm_serializer(X_test[1000:1010].toarray()))

# print(result)
# result = fm_predictor.predict(X_test[1000:1010].toarray())
print (Y_test[1000:1010])

{'instances': [{'features': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (400) from model with message "unable to evaluate payload provided". See https://us-west-2.console.aws.amazon.com/cloudwatch/home?region=us-west-2#logEventViewer:group=/aws/sagemaker/Endpoints/factorization-machines-2020-12-08-02-41-09-967 in account 517141035927 for more information.

In [ ]:
result = fm_predictor.predict(X_test[1000:1010].toarray())
print(result)
print (Y_test[1000:1010])

In [43]:
import boto3
import time

client = boto3.client("sagemaker-runtime")

endpoint_name = "factorization-machines-2020-12-08-02-41-09-967"
content_type="application/json"
accept="application/json"

# result = fm_predictor.predict(X_test[1000:1010].toarray())
# print(result)
# print (Y_test[1000:1010])
payload=X_test[1000:1010].toarray()

print(payload)

response = client.invoke_endpoint(
    EndpointName=endpoint_name, 
    ContentType=content_type,
    Accept=accept,
    Body=payload
    )
    
print(response)


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


ParamValidationError: Parameter validation failed:
Invalid type for parameter Body, value: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], type: <class 'numpy.ndarray'>, valid types: <class 'bytes'>, <class 'bytearray'>, file-like object